In [45]:
import json

ip_gold = "/Users/milind/Desktop/Milind/UTD/Summer 23/Qualification Annotation Generation/Experiments/template_based/predictions_extractions.json"
ip_kairos= "event_role_KAIROS.json"

In [50]:
with open(ip_gold, 'r') as json_file:
    gold_data = json.load(json_file)
with open(ip_kairos, 'r') as json_file2:
    kairos_role = json.load(json_file2)    

In [47]:
kairos_role["ArtifactExistence.DamageDestroyDisableDismantle.Damage"]["template"]

'<arg1> damaged <arg2> using <arg3> instrument in <arg4> place'

In [48]:
gold_data["wiki_mass_car_bombings_1_news_8"]["doc_id"]

TypeError: list indices must be integers or slices, not str

In [51]:
total_role_types={}
for key,value in kairos_role.items():
    for i in range(len(value['roles'])):
        if(value['roles'][i] not in total_role_types.keys()):
            total_role_types[value['roles'][i]]=set()
        total_role_types[value['roles'][i]].update(value['role_types'][i])

In [52]:
print(len(total_role_types))

80


In [53]:
question_map = {}
for key, value in total_role_types.items():
    if ("per" in value or "org" in value):
        if("Who" not in question_map.keys()):
            question_map["Who"]=[]
        question_map["Who"].append(key)
    elif ("per" in value or "org" in value):
        if("Whom" not in question_map.keys()):
            question_map["Whom"]=[]
        question_map["Whom"].append(key)
    elif ("loc" in value or "fac" in value):
        if("Where" not in question_map.keys()):
            question_map["Where"]=[]
        question_map["Where"].append(key)
    elif ("mhi" in value):
        if("Why" not in question_map.keys()):
            question_map["Why"]=[]
        question_map["Why"].append(key)
    elif ("mhi" in value):
        if("How" not in question_map.keys()):
            question_map["How"]=[]
        question_map["How"].append(key)
#     elif ("loc" in value || "fac" in value):
#         question_map["When"].update(key)
    else:
        if("What" not in question_map.keys()):
            question_map["What"]=[]
        question_map["What"].append(key)
    

In [54]:
op_dictionary={}

In [55]:
def find_wh(role):
    wh_list=[]
    for k,v in question_map.items():
        if(role in v):
            wh_list.append(k) 
#     print(wh_list)
    return wh_list

In [57]:
#FOR PREDICTED EXATRACTIONS
for item in gold_data:
    op_dictionary[item["doc_id"]]={"events":[]}
    for event in item["event_mentions"]:
        temp_dict={}
        event_id=event["id"]
        trigger = event["trigger"]["text"]
        qa_list=[]
        for arg in event["arguments"]:
            arg_role=arg['role']
            arg_text=arg['text']
#             print(arg_role)
            wh = find_wh(arg_role)
            for wh_word in wh:
                question = wh_word + ' ' + trigger + '?'
                answer = arg_text
                qa_dict={'question': question,'answer': answer}
                qa_list.append(qa_dict)
        temp_dict[event_id]=qa_list
        op_dictionary[item["doc_id"]]["events"].append(temp_dict)

In [40]:
#For GOLD EXTRACTIONS
for key, value in gold_data.items():
    op_dictionary[key]={"events":[]}
    for item in value["event_mentions"]:
        temp_dict={}
        event_id=item["id"]
        trigger = item["trigger"]["text"]
        qa_list=[]
        for arg in item["arguments"]:
            arg_role=arg['role']
            arg_text=arg['text']
#             print(arg_role)
            wh = find_wh(arg_role)
            for wh_word in wh:
                question = wh_word + ' ' + trigger + '?'
                answer = arg_text
                qa_dict={'question': question,'answer': answer}
                qa_list.append(qa_dict)
        temp_dict[event_id]=qa_list
        op_dictionary[key]["events"].append(temp_dict)

In [58]:
print(op_dictionary)

{'wiki_mass_car_bombings_1_news_8': {'events': [{'wiki_mass_car_bombings_1_news_8-E1': [{'question': 'Who killed?', 'answer': 'Taliban'}, {'question': 'Who killed?', 'answer': 'members'}]}, {'wiki_mass_car_bombings_1_news_8-E2': [{'question': 'Where explosion?', 'answer': 'training center'}]}, {'wiki_mass_car_bombings_1_news_8-E3': [{'question': 'Who killed?', 'answer': 'people'}]}, {'wiki_mass_car_bombings_1_news_8-E4': [{'question': 'Where the assault?', 'answer': 'campus'}, {'question': 'Who the assault?', 'answer': 'attackers'}]}, {'wiki_mass_car_bombings_1_news_8-E5': [{'question': 'Where detonated?', 'answer': 'campus'}, {'question': 'Who detonated?', 'answer': 'attackers'}]}, {'wiki_mass_car_bombings_1_news_8-E6': [{'question': 'Where shot?', 'answer': 'campus'}, {'question': 'Who shot?', 'answer': 'soldiers'}, {'question': 'Who shot?', 'answer': 'gunmen'}]}, {'wiki_mass_car_bombings_1_news_8-E7': [{'question': 'Who killed?', 'answer': 'members'}]}, {'wiki_mass_car_bombings_1_ne

In [59]:
pretty_json = json.dumps(op_dictionary, indent=4)
output_file_path = 'Rule_BasedQG_pred.json'
with open(output_file_path, 'w') as output_file:
    output_file.write(pretty_json)
print(f"Data saved to {output_file_path}")

Data saved to Rule_BasedQG_pred.json


In [61]:
import nltk
from nltk.tokenize import sent_tokenize
# Input text
input_text = "A settlement has been reached in a $1-million lawsuit filed by a taxi driver accusing police of negligence after he got caught up in the August 2016 take-down of ISIS-sympathizer Aaron Driver. READ MORE: FBI agent whose tip thwarted 2016 ISIS <b>attack</b> in Ontario says he was glad to help. Terry Duffield was injured when Driver detonated a homemade explosive in the back of his cab in August 2016. “I have to be very careful because there is an agreement to not disclose any of the terms of the settlement,” Duffield’s lawyer Kevin Egan told 980 CFPL."

# Target word to locate
target_word = "attack"

# Tokenize the text into sentences using full stops as separators
sentences = input_text.split('. ')

# Find the sentence containing the target word and extract the adjacent sentences
target_sentence = None
for i, sentence in enumerate(sentences):
    if target_word in sentence:
        target_sentence = sentence
        before_sentence = sentences[i - 1] if i > 0 else None
        after_sentence = sentences[i + 1] if i < len(sentences) - 1 else None
        break

# Print the results
print("Target Sentence:")
print(target_sentence)

if before_sentence:
    print("\nSentence Before:")
    print(before_sentence)

if after_sentence:
    print("\nSentence After:")
    print(after_sentence)



Target Sentence:
READ MORE: FBI agent whose tip thwarted 2016 ISIS <b>attack</b> in Ontario says he was glad to help

Sentence Before:
A settlement has been reached in a $1-million lawsuit filed by a taxi driver accusing police of negligence after he got caught up in the August 2016 take-down of ISIS-sympathizer Aaron Driver

Sentence After:
Terry Duffield was injured when Driver detonated a homemade explosive in the back of his cab in August 2016


In [64]:
type(target_sentence)

str